In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_parquet("train_data.pq")
sample = pd.read_csv("sample_submission.csv")

In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np

class ConfigurableRecommender:
    def __init__(self, config):
        self.config = config
        self.train = None
        self.sample = None
        self.prepared_data = {}
        
    def load_data(self, train_path, sample_path):
        """Загрузка данных"""
        self.train = pd.read_parquet(train_path)
        self.sample = pd.read_csv(sample_path)
        return self
    
    def prepare_data(self):
        """Предварительная подготовка данных согласно конфигу"""
        print("Подготавливаем данные...")
        
        last_day = self.train["date"].max()
        self.prepared_data['last_day'] = last_day
        k = self.config['time_window']['k']
        
        # Подготовка данных для каждого источника
        for source_name, source_config in self.config['sources'].items():
            if not source_config['enabled']:
                continue
                
            if source_name == 'multiple_clicks_last_day':
                self._prepare_multiple_clicks_last_day(last_day)
                
            elif source_name == 'last_day_clicks':
                self._prepare_last_day_clicks(last_day)
                
            elif source_name == 'previous_days':
                self._prepare_previous_days(last_day, k)
                
            elif source_name == 'global_popular':
                self._prepare_global_popular(last_day)
        
        return self
    
    def _prepare_multiple_clicks_last_day(self, last_day):
        """Подготовка данных по множественным кликам в последний день"""
        last_day_data = self.train[self.train["date"] == last_day]
        user_multiple_clicks = {}
        
        last_day_counts = last_day_data.groupby(["user_id", "item_id"]).size()
        for (user_id, item_id), count in last_day_counts.items():
            if count > self.config['sources']['multiple_clicks_last_day']['min_clicks']:
                if user_id not in user_multiple_clicks:
                    user_multiple_clicks[user_id] = []
                user_multiple_clicks[user_id].append(item_id)
        
        self.prepared_data['multiple_clicks_last_day'] = user_multiple_clicks
        print(f"  multiple_clicks_last_day: {len(user_multiple_clicks)} пользователей")
    
    def _prepare_last_day_clicks(self, last_day):
        """Подготовка данных по всем кликам в последний день"""
        last_day_data = self.train[self.train["date"] == last_day]
        user_last_clicks = last_day_data.groupby("user_id")["item_id"].apply(list).to_dict()
        self.prepared_data['last_day_clicks'] = user_last_clicks
        print(f"  last_day_clicks: {len(user_last_clicks)} пользователей")
    
    def _prepare_previous_days(self, last_day, k):
        """Подготовка данных по предыдущим дням"""
        previous_days_candidates = {}
        thresholds = self.config['sources']['previous_days']['thresholds']
        
        for day_offset in range(1, k + 1):
            target_date = last_day - day_offset
            day_data = self.train[self.train['date'] == target_date]
            
            if not day_data.empty:
                threshold = thresholds[day_offset - 1] if day_offset <= len(thresholds) else thresholds[-1]
                
                day_counts = day_data.groupby(['user_id', 'item_id']).size().reset_index(name='count')
                valid_items = day_counts[day_counts['count'] >= threshold]
                
                for _, row in valid_items.iterrows():
                    user_id = row['user_id']
                    item_id = row['item_id']
                    
                    if user_id not in previous_days_candidates:
                        previous_days_candidates[user_id] = []
                    previous_days_candidates[user_id].append(item_id)
        
        self.prepared_data['previous_days'] = previous_days_candidates
        print(f"  previous_days: {len(previous_days_candidates)} пользователей")
    
    def _prepare_global_popular(self, last_day):
        """Подготовка глобально популярных товаров"""
        if self.config['sources']['global_popular']['use_recent_days']:
            # Используем несколько последних дней для популярности
            recent_days = self.config['sources']['global_popular']['recent_days']
            recent_data = self.train[self.train['date'] >= (last_day - recent_days)]
            global_popular = recent_data['item_id'].value_counts().head(100).index.tolist()
        else:
            # Только последний день
            last_day_data = self.train[self.train["date"] == last_day]
            global_popular = last_day_data["item_id"].value_counts().head(100).index.tolist()
        
        self.prepared_data['global_popular'] = global_popular
        print(f"  global_popular: {len(global_popular)} товаров")
    
    def recommend_for_user(self, uid):
        """Генерация рекомендаций для одного пользователя"""
        recommendations = []
        used_items = set()
        
        # Проходим по всем источникам в порядке приоритета
        for source_name in self.config['priority_order']:
            source_config = self.config['sources'][source_name]
            
            if not source_config['enabled'] or len(recommendations) >= 20:
                continue
            
            max_candidates = source_config['max_candidates']
            candidates = self._get_candidates_from_source(uid, source_name)
            
            # Добавляем кандидатов, пока не достигнем лимита
            remaining_slots = min(20 - len(recommendations), max_candidates)
            for item in candidates:
                if item not in used_items and remaining_slots > 0:
                    recommendations.append(item)
                    used_items.add(item)
                    remaining_slots -= 1
                if len(recommendations) >= 20:
                    break
        
        # Гарантируем 20 рекомендаций
        if len(recommendations) < 20:
            self._fill_remaining_slots(recommendations, used_items)
        
        return recommendations[:20]
    
    def _get_candidates_from_source(self, uid, source_name):
        """Получение кандидатов из указанного источника"""
        if source_name == 'multiple_clicks_last_day':
            return self.prepared_data.get('multiple_clicks_last_day', {}).get(uid, [])
        
        elif source_name == 'last_day_clicks':
            return self.prepared_data.get('last_day_clicks', {}).get(uid, [])
        
        elif source_name == 'previous_days':
            return self.prepared_data.get('previous_days', {}).get(uid, [])
        
        elif source_name == 'global_popular':
            return self.prepared_data.get('global_popular', [])
        
        return []
    
    def _fill_remaining_slots(self, recommendations, used_items):
        """Заполнение оставшихся слотов глобально популярными товарами"""
        global_popular = self.prepared_data.get('global_popular', [])
        remaining_slots = 20 - len(recommendations)
        
        for item in global_popular:
            if item not in used_items and remaining_slots > 0:
                recommendations.append(item)
                used_items.add(item)
                remaining_slots -= 1
            if remaining_slots <= 0:
                break
        
        # Если все еще не хватает, используем самые популярные из всего датасета
        if remaining_slots > 0:
            all_popular = self.train['item_id'].value_counts().head(100).index.tolist()
            for item in all_popular:
                if item not in used_items and remaining_slots > 0:
                    recommendations.append(item)
                    used_items.add(item)
                    remaining_slots -= 1
                if remaining_slots <= 0:
                    break
    
    def generate_submission(self, output_path):
        """Генерация финального сабмита"""
        print("Генерация рекомендаций...")
        
        test_users = self.sample["user_id"].unique()
        recs = []
        
        for uid in tqdm(test_users):
            items = self.recommend_for_user(uid)
            for item in items:
                recs.append((uid, item))
        
        submission = pd.DataFrame(recs, columns=["user_id", "item_id"])
        
        # Проверка корректности
        user_counts = submission['user_id'].value_counts()
        if not (user_counts == 20).all():
            print("Исправляем пользователей без 20 рекомендаций...")
            submission = self._fix_submission(submission)
        
        submission.to_csv(output_path, index=False)
        print(f"✅ Сабмит сохранен: {output_path}")
        print(f"   Всего рекомендаций: {len(submission)}")
        print(f"   Уникальных пользователей: {submission['user_id'].nunique()}")
        print(f"   Уникальных товаров: {submission['item_id'].nunique()}")
        
        return submission
    
    def _fix_submission(self, submission):
        """Исправление сабмита для пользователей без 20 рекомендаций"""
        global_popular = self.prepared_data.get('global_popular', [])
        fixed_recs = []
        
        for uid in self.sample["user_id"].unique():
            user_items = submission[submission['user_id'] == uid]['item_id'].tolist()
            
            if len(user_items) < 20:
                # Добираем недостающие глобально популярными
                additional_items = [item for item in global_popular if item not in user_items]
                user_items.extend(additional_items[:20 - len(user_items)])
            
            # Гарантируем ровно 20
            for item in user_items[:20]:
                fixed_recs.append((uid, item))
        
        return pd.DataFrame(fixed_recs, columns=["user_id", "item_id"])

In [15]:
# # Конфиг 3: Сбалансированный (рекомендуемый)
CONFIG_BALANCED = {
    'time_window': {
        'k': 12
    },
    'sources': {
        'multiple_clicks_last_day': {
            'enabled': True,
            'min_clicks': 1,
            'max_candidates': 20
        },
        'last_day_clicks': {
            'enabled': True,
            'max_candidates': 20
        },
        'previous_days': {
            'enabled': True,
            'thresholds': [1]*11,
            'max_candidates': 20
        },
        'global_popular': {
            'enabled': True,
            'use_recent_days': True,
            'recent_days': 1,
            'max_candidates': 20
        }
    },
    'priority_order': [
        'multiple_clicks_last_day',
        'last_day_clicks',
        'previous_days',
        'global_popular'
    ]
}

In [16]:
# ==================== ИСПОЛЬЗОВАНИЕ ====================

if __name__ == "__main__":
    # Выберите конфиг для эксперимента
    CURRENT_CONFIG = CONFIG_BALANCED
    
    # Инициализация и запуск
    recommender = ConfigurableRecommender(CURRENT_CONFIG)
    recommender.load_data(
        "train_data.pq",
        "sample_submission.csv"
    )
    
    # Подготовка данных
    recommender.prepare_data()
    
    # Генерация сабмита
    submission = recommender.generate_submission("bilibert.csv")
    
    print("\n🎯 Конфигурация использована:")
    for source in CURRENT_CONFIG['priority_order']:
        if CURRENT_CONFIG['sources'][source]['enabled']:
            config = CURRENT_CONFIG['sources'][source]
            print(f"   - {source}: max_{config['max_candidates']}")

Подготавливаем данные...
  multiple_clicks_last_day: 0 пользователей
  last_day_clicks: 98917 пользователей
  previous_days: 962600 пользователей
  global_popular: 100 товаров
Генерация рекомендаций...


100%|██████████| 293230/293230 [00:03<00:00, 73509.32it/s]


✅ Сабмит сохранен: bilibert.csv
   Всего рекомендаций: 5864600
   Уникальных пользователей: 293230
   Уникальных товаров: 135243

🎯 Конфигурация использована:
   - multiple_clicks_last_day: max_20
   - last_day_clicks: max_20
   - previous_days: max_20
   - global_popular: max_20


In [23]:
check = pd.read_csv('da_bilo_x2.csv')
f = list(submission['item_id'])
s = list(check['item_id'])

In [24]:
eq = 0
for i in range(len(f)):
    if f[i] == s[i]:
        eq += 1
print(eq)

5407810


In [22]:
len(f)

5864600